In [1]:
import sys
import os
home=os.getcwd()
sys.path.append(home+'/Functions')
from ChargeDataSet import *
from MS_L_IDs import *
from ShowDF import *
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from AllMS2Data import *
from feat_ms2_Gauss import *

In [14]:
AligningFolder = 'Alignedms2Features_254-255_20260129_afterCodeChanges'
FeaturesClusters = os.listdir(AligningFolder)
FeaturesClustersList = []
FeatureClustersTest = []
#FeaturesClusters_toConsider = list(AlignedSamplesDF['feat_id'][[0,1,2,3]])
for feature_cluster in FeaturesClusters:
#for feature_cluster in [207,208,213,214]:
    print(feature_cluster)
    FeatureClusterLoc = AligningFolder +'/'+str(feature_cluster)#+'.csv'
    FeaturesClusterDF = pd.read_csv(FeatureClusterLoc,index_col = 0)
    FeatureClustersTest += np.array(FeaturesClusterDF).tolist()
    FeaturesClustersList.append(FeaturesClusterDF)
    ShowDF(FeaturesClusterDF)

9.csv


,ms2_id,Sample_id
0,2454,42
1,2465,43
2,2455,44
3,2767,4
4,2496,45
5,2464,46
6,2802,5
7,2474,12
8,2364,14
9,2624,20


19.csv


,ms2_id,Sample_id
0,2520,37
1,2557,19
2,2490,38
3,2625,20
4,2467,23
5,2466,43
6,2583,24
7,2414,7
8,2456,44
9,2441,8


5.csv


...

,ms2_id,Sample_id
0,2629,18
1,2558,19
2,2522,37
3,2491,38
4,2491,22
5,2584,24
6,2415,7
7,2457,44
8,2442,8
9,2498,45


4.csv


,ms2_id,Sample_id
0,2365,39
1,2519,21
2,2468,23
3,2457,26
4,2475,12


10.csv


,ms2_id,Sample_id
0,2453,42
1,2749,3
2,2464,43
3,2488,22
4,2494,47
5,2806,5
6,2503,9
7,2573,36
8,2365,14
9,2625,18


In [11]:
DataFolder=home+'/ms2_spectra_Summ-20260107'
SamplesList = os.listdir(DataFolder)
SamplesList.sort()
SamplesNames = [sample_name.replace('mzML-ms2Summary.xlsx','') for sample_name in SamplesList]

from Retrieve_and_Join_ms2_for_feature import *
ResultsFolder = home+'/ms2_spectra_Summ-20260107'
ms2Folder = home+'/ms2_spectra-20260107'
All_ms2,Spectra_idVec = Retrieve_and_Join_ms2_for_feature(All_FeaturesTable = np.array(FeatureClustersTest),
                                                          Feature_module =  np.arange(N_spectra),
                                                          SamplesNames = SamplesNames,
                                                          sample_id_col = 1,
                                                          ms2_spec_id_col = 0,
                                                          ms2Folder = ms2Folder,
                                                          ToAdd = 'mzML',
                                                          Norm2One = True)

In [18]:
N_spectra

7

In [22]:
N_spectra = len(FeatureClustersTest)

In [25]:
Feature_module =  np.arange(N_spectra)
Feature_module

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134])

In [200]:
from Retrieve_and_Join_ms2_for_feature import *
ResultsFolder = home+'/ms2_spectra_Summ-20260107'
ms2Folder = home+'/ms2_spectra-20260107'
All_ms2,Spectra_idVec = Retrieve_and_Join_ms2_for_feature(All_FeaturesTable = np.array(FeatureClustersTest),
                                                          Feature_module =  np.arange(N_spectra),
                                                          SamplesNames = SamplesNames,
                                                          sample_id_col = 1,
                                                          ms2_spec_id_col = 0,
                                                          ms2Folder = ms2Folder,
                                                          ToAdd = 'mzML',
                                                          Norm2One = True)

In [27]:
from AdjacencyList_ms2Fragments import *
from ms2_feat_modules import *
from CosineMatrix import *
AdjacencyListFragments,feat_ids = AdjacencyList_ms2Fragments(All_ms2 = All_ms2)
Feature_module =  np.arange(N_spectra)
N_features = len(Feature_module)
Frag_Modules = ms2_feat_modules(AdjacencyList = AdjacencyListFragments,
                                ms2_ids = feat_ids)
AlignedFragmentsMat,AlignedFragments_mz_Mat = AligniningFragments_in_Feature(Frag_Modules = Frag_Modules,
                                                                             All_ms2 = All_ms2,
                                                                             N_features = N_features)    
ShowDF(AlignedFragmentsMat)
AlignedFragmentsMat,AlignedFragments_mz_Mat = minimalAlignedFragmentsMat(AlignedFragmentsMat = AlignedFragmentsMat,
                                                                         AlignedFragments_mz_Mat = AlignedFragments_mz_Mat,
                                                 Intensity_to_explain = 0.9,
                                                 min_spectra = 3)
ShowDF(AlignedFragmentsMat)
CosineMat = CosineMatrix(AlignedFragmentsMat = AlignedFragmentsMat,
                         N_features = N_features)
ShowDF(CosineMat)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135
0,55.0542,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00283075,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,83.0855,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0232365,0.0228841,0.02698,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00814798,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0359021,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,89.0599,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0592652,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,92.0494,0.090239,0.0849794,0.0708033,0.0931118,0.0814,0.0661321,0.0685449,0.0888447,0.0986935,0.0762792,0.0831158,0.0758492,0.102893,0.0939572,0.0720235,0.0562226,0.0618182,0,0.071915,0,0.0610281,0.0878987,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0782167,0.0665849,0.064665,0.0695608,0.126053,0.0649184,0.0674471,0.109461,0.0629225,0.076716,0.0659599,0.0735124,0.0722373,0.076649,0.0643527,0.0710751,0.0653853,0.0625215,0.072537,0.0860477,0.0735409,0.064738,0.0887784,0.0708275,0.0609024,0.0650199,0.0591729,0.0629098,0.0682095,0.0773202,0.0753091,0.0603633,0.0861576,0.0650115,0.0618845,0.0729221,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.07945,0.0630933,0.0660582,0.075863,0.0643701,0.0609969,0.0622969,0.0655861,0.106403,0.074107
4,93.0574,0.0510359,0.0591371,0.0380464,0.0630814,0.0511349,0.05438,0.0396287,0.0480773,0,0,0,0.0541143,0.0547259,0.0374853,0.0583491,0.0509379,0,0,0,0,0.0571294,0.0478944,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0558354,0.0558368,0.0580273,0.0523071,0,0.0525925,0.054861,0,0.0494397,0.0638292,0.0503969,0.052816,0.0587787,0.0530345,0.0434679,0.0563732,0.0553464,0.0491609,0.0566848,0.0553388,0.0568474,0.0517653,0.051218,0.0501379,0.0577638,0.0523474,0.0585649,0.0569692,0.0548008,0.042004,0.0567826,0.0530797,0.0584728,0.0550342,0.0489815,0.0623166,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.054654,0.058197,0.0569296,0.0494821,0.0565216,0.0484509,0.0558638,0.0551754,0
5,94.0651,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.008303,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,94.458,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0032324,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,99.0553,0.0521297,0.0729614,0.0678782,0.0729427,0.0455719,0.0446937,0.0551626,0.0578,0,0,0,0.0478194,0.0631596,0.0726069,0.066879,0.0512618,0.077649,0,0.0641882,0,0.0782004,0.0480595,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0695411,0.071333,0.066247,0.0771982,0.104743,0.0568803,0.061759,0.113464,0.0721912,0.0614093,0.0690343,0.0642885,0.0728765,0.0547081,0.0688797,0.0604512,0.0587975,0.0648334,0.0686267,0.0863933,0.0665977,0.0638196,0.0764575,0.0697795,0.0727719,0.0659367,0.0638546,0.0728443,0.0678584,0.0500166,0.0

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135
0,92.0494,0.090239,0.0849794,0.0708033,0.0931118,0.0814,0.0661321,0.0685449,0.0888447,0.0986935,0.0762792,0.0831158,0.0758492,0.102893,0.0939572,0.0720235,0.0562226,0.0618182,0,0.071915,0,0.0610281,0.0878987,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0782167,0.0665849,0.064665,0.0695608,0.126053,0.0649184,0.0674471,0.109461,0.0629225,0.076716,0.0659599,0.0735124,0.0722373,0.076649,0.0643527,0.0710751,0.0653853,0.0625215,0.072537,0.0860477,0.0735409,0.064738,0.0887784,0.0708275,0.0609024,0.0650199,0.0591729,0.0629098,0.0682095,0.0773202,0.0753091,0.0603633,0.0861576,0.0650115,0.0618845,0.0729221,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.07945,0.0630933,0.0660582,0.075863,0.0643701,0.0609969,0.0622969,0.0655861,0.106403,0.074107
1,93.0574,0.0510359,0.0591371,0.0380464,0.0630814,0.0511349,0.05438,0.0396287,0.0480773,0,0,0,0.0541143,0.0547259,0.0374853,0.0583491,0.0509379,0,0,0,0,0.0571294,0.0478944,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0558354,0.0558368,0.0580273,0.0523071,0,0.0525925,0.054861,0,0.0494397,0.0638292,0.0503969,0.052816,0.0587787,0.0530345,0.0434679,0.0563732,0.0553464,0.0491609,0.0566848,0.0553388,0.0568474,0.0517653,0.051218,0.0501379,0.0577638,0.0523474,0.0585649,0.0569692,0.0548008,0.042004,0.0567826,0.0530797,0.0584728,0.0550342,0.0489815,0.0623166,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.054654,0.058197,0.0569296,0.0494821,0.0565216,0.0484509,0.0558638,0.0551754,0
2,99.0553,0.0521297,0.0729614,0.0678782,0.0729427,0.0455719,0.0446937,0.0551626,0.0578,0,0,0,0.0478194,0.0631596,0.0726069,0.066879,0.0512618,0.077649,0,0.0641882,0,0.0782004,0.0480595,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0695411,0.071333,0.066247,0.0771982,0.104743,0.0568803,0.061759,0.113464,0.0721912,0.0614093,0.0690343,0.0642885,0.0728765,0.0547081,0.0688797,0.0604512,0.0587975,0.0648334,0.0686267,0.0863933,0.0665977,0.0638196,0.0764575,0.0697795,0.0727719,0.0659367,0.0638546,0.0728443,0.0678584,0.0500166,0.0679105,0.0615849,0.0715912,0.0610049,0.0739526,0.0683251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0671676,0.064262,0.0555431,0.0657013,0.0615601,0.0637725,0.0748767,0.0727683,0
3,108.044,0.113258,0.10433,0.114139,0.129864,0.136532,0.147264,0.104496,0.128889,0.108432,0.136473,0.118095,0.0972221,0.103973,0.11437,0.112295,0.101851,0.130728,0,0.114614,0.15983,0.145143,0.0893025,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.130418,0.112119,0.126492,0.118722,0.136714,0.122502,0.114551,0.125036,0.11036,0.0986815,0.116141,0.122153,0.132572,0.136697,0.123568,0.11754,0.101472,0.117976,0.120403,0.097957,0.125265,0.118961,0.155207,0.111818,0.123156,0.133099,0.128222,0.118891,0.118337,0.114,0.111369,0.124609,0.101377,0.113404,0.116024,0.126214,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.111721,0.12051,0.119859,0.120699,0.133822,0.121809,0.0922543,0.110897,0.130673,0.0829982
4,128.107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0501972,0.0636748,0.0688585,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0183817,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,138.091,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.761072,0.77218,0.747461,0.773517,0.829906,0.818215,0.808489,0.738171,0.789375,0.797157,0.759977,0.731581,0.796018,0.812658,0.755181,0.83429,0.790171,

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134
0,1,0.992085,0.996473,0.98547,0.985248,0.993353,0.990401,0.992313,0.955712,0.956849,0.977436,0.992097,0.988966,0.986391,0.990178,0.985719,0.980255,0.944735,0.986556,0.950692,0.990178,0.986154,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.977628,0.982538,0.982082,0.98063,0.965991,0.977915,0.980382,0.95475,0.978421,0.982128,0.98136,0.981371,0.978347,0.973533,0.969866,0.979541,0.942988,0.973893,0.978352,0.958926,0.971483,0.981028,0.969317,0.971809,0.975638,0.982512,0.979332,0.980534,0.979228,0.97203,0.979903,0.981334,0.970526,0.975562,0.978681,0.982811,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.967769,0.987518,0.976094,0.987248,0.977805,0.987067,0.98872,0.985348,0.986188,0.967476
1,0.992085,1,0.99553,0.982611,0.986381,0.99323,0.989875,0.991758,0.936113,0.922685,0.989011,0.996121,0.991033,0.990145,0.991911,0.992371,0.991816,0.926426,0.990287,0.931191,0.995876,0.992251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.985475,0.988492,0.990562,0.987962,0.984732,0.985359,0.986848,0.978134,0.985806,0.986808,0.988306,0.987853,0.989554,0.979648,0.982439,0.984898,0.961403,0.9831,0.987516,0.972957,0.981546,0.987799,0.975405,0.985608,0.983574,0.987265,0.986397,0.988546,0.986674,0.982979,0.987539,0.98577,0.983273,0.984996,0.986051,0.989287,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.984502,0.990403,0.985153,0.989871,0.984894,0.993047,0.990021,0.990262,0.994849,0.984399
2,0.996473,0.99553,1,0.987542,0.991255,0.996519,0.995353,0.997023,0.932428,0.934198,0.983818,0.998358,0.995247,0.991356,0.995791,0.993256,0.986748,0.92304,0.98537,0.929416,0.997325,0.992029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.989555,0.992826,0.993859,0.991755,0.97839,0.989699,0.991235,0.971674,0.990681,0.991169,0.992448,0.991995,0.990834,0.985517,0.98488,0.990573,0.964194,0.987809,0.991619,0.973984,0.985743,0.992316,0.982376,0.987432,0.988414,0.992453,0.990795,0.992086,0.990713,0.986102,0.99142,0.991977,0.984947,0.988533,0.990462,0.993491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.979058,0.995125,0.986889,0.994315,0.987827,0.996636,0.995355,0.993957,0.994971,0.978738
3,0.98547,0.982611,0.987542,1,0.986733,0.989352,0.988942,0.990207,0.920818,0.924371,0.967638,0.985541,0.988926,0.977868,0.989484,0.985706,0.970926,0.90726,0.970261,0.916435,0.984723,0.985582,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.988572,0.985746,0.98095,0.984649,0.963804,0.983306,0.987213,0.956198,0.983122,0.985893,0.98461,0.985282,0.982073,0.981361,0.977271,0.984388,0.949143,0.980292,0.978858,0.967274,0.982456,0.984404,0.979797,0.973682,0.982231,0.986195,0.984087,0.984109,0.98409,0.977571,0.984003,0.985127,0.977708,0.981401,0.983721,0.985152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.962212,0.988422,0.972849,0.988026,0.97398,0.983806,0.987977,0.987396,0.982788,0.961251
4,0.985248,0.986381,0.991255,0.986733,1,0.994619,0.99501,0.996932,0.915842,0.921463,0.982488,0.990825,0.996845,0.989693,0.997994,0.997461,0.982943,0.90276,0.977728,0.912976,0.992299,0.99732,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.994018,0.996298,0.990766,0.996584,0.978533,0.99538,0.995429,0.973409,0.994098,0.99468,0.996436,0.996128,0.992873,0.991653,0.993313,0.994892,0.971875,0.99316,0.988995,0.988966,0.9925,0.994644,0.992176,0.986097,0.994711,0.996375,0.995747,0.995467,0.994897,0.991162,0.995076,0.996376,0.988364,0.994692,0.995978,0.995536,0,0,0,0,0,0,0,0,0,0,0,0,0

In [28]:
AlignedFragmentsMat,AlignedFragments_mz_Mat = minimalAlignedFragmentsMat(AlignedFragmentsMat = AlignedFragmentsMat,
                                                                         AlignedFragments_mz_Mat = AlignedFragments_mz_Mat,
                                                 Intensity_to_explain = 0.9,
                                                 min_spectra = 3)

In [29]:
from AdjacencyList_from_matrix import *
from CommunityBlocks import *

AdjacencyList_Features,features_ids = AdjacencyList_from_matrix(CosineMat = CosineMat,
                                                                N_ms2_spectra = N_features,
                                                                cos_tol = 0.9)
Feature_Modules = CommunityBlocks(AdjacencyList_Features = AdjacencyList_Features)

In [30]:
from OverlappingClustering import *
IntramoduleSimilarity,CompactCosineTen,Modules,ConflictiveNeighborsList = OverlappingClustering(Feature_Modules,
                                                                                                CosineMat,
                                                                                                percentile = 10)
                                                                                                

In [2]:
import numpy as np
def AligniningFragments_in_Feature(Frag_Modules,
                                   All_ms2,
                                   N_features):
    N_Fragments = len(Frag_Modules)
    AlignedFragmentsMat = np.zeros((N_Fragments,
                                    N_features + 1))
    AlignedFragments_mz_Mat = np.zeros((N_Fragments,
                                        N_features + 1))
    for fragment_id in np.arange(N_Fragments,
                                 dtype = 'int'):
        Fragment_module = Frag_Modules[fragment_id]
        FragmentTable = All_ms2[Fragment_module, :]
        AlignedFragmentsMat[fragment_id, 0] = np.mean(FragmentTable[:, 0])
        AlignedFragments_mz_Mat[fragment_id, 0] = np.mean(FragmentTable[:, 0])
        Fragments_ids = np.array(FragmentTable[:, 10],
                                 dtype = 'int')
        AlignedFragmentsMat_loc = Fragments_ids + 1
        AlignedFragmentsMat[fragment_id, AlignedFragmentsMat_loc] = FragmentTable[:, 9]
        AlignedFragments_mz_Mat[fragment_id, AlignedFragmentsMat_loc] = FragmentTable[:, 0]
    FragmentsOrder = AlignedFragmentsMat[:, 0].argsort()
    AlignedFragmentsMat = AlignedFragmentsMat[FragmentsOrder]
    AlignedFragments_mz_Mat = AlignedFragments_mz_Mat[FragmentsOrder]
    return [AlignedFragmentsMat, AlignedFragments_mz_Mat]

import numpy as np
from minimalSpectrum import *
def minimalAlignedFragmentsMat(AlignedFragmentsMat,
                               AlignedFragments_mz_Mat,
                               Intensity_to_explain = 0.9,
                               min_spectra = 3):    
    BinaryAlignedFragmentsMat = (AlignedFragmentsMat[:,1:] > 0).astype(int)
    FragmentAbundanceVec = np.sum(BinaryAlignedFragmentsMat, axis = 1)
    IntensityContributionVec = np.sum(AlignedFragmentsMat[:,1:], axis = 0)
    N_spectra = BinaryAlignedFragmentsMat.shape[1]
    Fragments_to_consider = []
    for spectrum_id in np.arange(N_spectra):
        FragmentsVec = AlignedFragmentsMat[:, spectrum_id+1]
        RareFragmentsList, FragmentsList, fractionIntensity = minimalSpectrum(FragmentsVec = FragmentsVec,
                                                                              FragmentAbundanceVec = FragmentAbundanceVec,
                                                                              min_spectra = min_spectra,
                                                                              fractionIntensity = 0,
                                                                              RareFragmentsList = [],
                                                                              FragmentsList = [],
                                                                              min_fractionIntensity = Intensity_to_explain*IntensityContributionVec[spectrum_id])
        Fragments_to_consider += FragmentsList
    Fragments_to_consider = list(set(Fragments_to_consider))
    Fragments_to_consider.sort()
    return [AlignedFragmentsMat[Fragments_to_consider, :], AlignedFragments_mz_Mat[Fragments_to_consider, :]]


In [3]:
from ttest import *
from scipy import stats
import numpy as np
def ConsensusFragment(SpectraFragmentVec,
                      SpectraIntensityVec,
                      consensus_spectra,
                      percentile_mz = 5,
                      percentile_Int = 10,
                      alpha = 0.01,
                      minSpectra = 3):
    SpectraFragmentLoc = np.where(SpectraFragmentVec > 0)
    N_spectra = len(SpectraFragmentVec[SpectraFragmentLoc])
    if N_spectra < minSpectra:
        return consensus_spectra
    mz = np.median(SpectraFragmentVec[SpectraFragmentLoc])
    min_mz = np.percentile(SpectraFragmentVec[SpectraFragmentLoc],
                           percentile_mz)
    max_mz = np.percentile(SpectraFragmentVec[SpectraFragmentLoc],
                           100 - percentile_mz)  
    Q1_mz = np.percentile(SpectraFragmentVec[SpectraFragmentLoc],
                          25)
    Q3_mz = np.percentile(SpectraFragmentVec[SpectraFragmentLoc],
                          75)    
    IQR_mz = Q3_mz - Q1_mz
    mz_std = 0#np.std(SpectraFragmentVec[SpectraFragmentLoc])
    p = 0#stat, p = stats.shapiro(SpectraFragmentVec[SpectraFragmentLoc])
    t_ref = 0#ttest(Nsignals = N_spectra,alpha = alpha)
    mz_CI = 0#t_ref * mz_std / np.sqrt(N_spectra)
    mz_CIppm = 0#mz_CI / mz * 1e6       
    median_Int = np.median(SpectraIntensityVec) 
    mean_Int = np.mean(SpectraIntensityVec) 
    min_Int = np.percentile(SpectraIntensityVec,
                            percentile_Int)
    max_Int = np.percentile(SpectraIntensityVec,
                            100 - percentile_Int)
    Q1_Int = np.percentile(SpectraIntensityVec,
                            25)
    Q3_Int = np.percentile(SpectraIntensityVec,
                            75)    
    consensus_spectra.append([mz,
                              mz_std,
                              p,
                              N_spectra,
                              mz_CI,
                              mz_CIppm,
                              mean_Int,
                              min_Int,
                              max_Int,
                              min_mz,
                              max_mz,
                              IQR_mz / mz * 1e6,
                              Q1_mz,
                              Q3_mz,
                              median_Int,
                              Q1_Int,
                              Q3_Int])    
    return consensus_spectra

In [4]:
import numpy as np
def MostIntenseFragmentNorm(consensus_spectra):
    consensus_spectra = np.array(consensus_spectra)
    maxInt = np.max(consensus_spectra[:, 6])
    maxIntFragLoc = np.where(consensus_spectra[:, 6] == maxInt)
    if len(maxIntFragLoc) > 1:
        maxIntFragLoc = maxIntFragLoc[0][0]
    else: 
        maxIntFragLoc = maxIntFragLoc[0]
    NormIntenseVec = 100 * consensus_spectra[:, 6] / consensus_spectra[maxIntFragLoc, 6]
    NormIntenseVec = NormIntenseVec.reshape(-1, 1)
    min_NormIntenseVec = 100 * consensus_spectra[:, 7] / consensus_spectra[maxIntFragLoc, 6]
    min_NormIntenseVec = min_NormIntenseVec.reshape(-1, 1)
    max_NormIntenseVec = 100 * consensus_spectra[:, 8] / consensus_spectra[maxIntFragLoc, 6]
    max_NormIntenseVec = max_NormIntenseVec.reshape(-1, 1)
    median_NormIntenseVec = 100 * consensus_spectra[:, 14] / consensus_spectra[maxIntFragLoc, 6]
    median_NormIntenseVec = median_NormIntenseVec.reshape(-1, 1)    
    NormIntenseVec_Q1 = 100 * consensus_spectra[:, 15] / consensus_spectra[maxIntFragLoc, 6]
    NormIntenseVec_Q1 = NormIntenseVec_Q1.reshape(-1, 1)
    NormIntenseVec_Q3 = 100 * consensus_spectra[:, 16] / consensus_spectra[maxIntFragLoc, 6]
    NormIntenseVec_Q3 = NormIntenseVec_Q3.reshape(-1, 1)    
    consensus_spectra = np.hstack((consensus_spectra, NormIntenseVec))
    consensus_spectra = np.hstack((consensus_spectra, min_NormIntenseVec))
    consensus_spectra = np.hstack((consensus_spectra, max_NormIntenseVec))
    consensus_spectra = np.hstack((consensus_spectra, median_NormIntenseVec))
    consensus_spectra = np.hstack((consensus_spectra, NormIntenseVec_Q1))
    consensus_spectra = np.hstack((consensus_spectra, NormIntenseVec_Q3))    
    return consensus_spectra

In [13]:
for x,y in [[1,2],[2,1],[4,5]]:
    print(x,y)

1 2
2 1
4 5


In [7]:
def UpdateModulesAfterClustering(New_Modules,
                                 Modules):
    if len(Modules) == len(New_Modules):
        return Modules
    UpdatedModules = []
    for new_module in New_Modules:
        updatedModule = []
        for module in new_module:
            updatedModule += Modules[module]
        UpdatedModules.append(updatedModule)
    return UpdatedModules

In [8]:
from CommunityBlocks import *
from IntramoduleSimilarityCalc import *
from CompressCosineMatrix import *
from AdjacentOverlappingModules import *
from CompactNeighbourhood import *
#from UpdateModulesAfterClustering import *
def OverlappingClustering(Feature_Modules,
                          CosineMat,
                          percentile = 10):
    Modules = Feature_Modules.copy()
    modulesDif = 42
    while modulesDif > 0:
        IntramoduleSimilarity = IntramoduleSimilarityCalc(Modules = Modules,
                                                          CosineMat = CosineMat.copy(),
                                                          percentile = percentile)
        CompactCosineTen = CompressCosineMatrix(Modules = Modules,
                                                CosineMat = CosineMat.copy(),
                                                percentile = percentile)
        AdjacencyList,AdjacencyMatrix,module_ids = AdjacentOverlappingModules(IntramoduleSimilarity = IntramoduleSimilarity,
                                                                              CompactCosineTen = CompactCosineTen)
        NewAdjacencyList,ConflictiveNeighborsList = CompactNeighbourhood(AdjacencyList = AdjacencyList.copy(),
                                                                         AdjacencyMatrix = AdjacencyMatrix.copy(),
                                                                         CosineMat = CompactCosineTen[:,:,1])        
        New_Modules = CommunityBlocks(AdjacencyList_Features = NewAdjacencyList,
                                      Order = -1)
        modulesDif = len(Modules) - len(New_Modules)        
        Modules = UpdateModulesAfterClustering(New_Modules = New_Modules,
                                               Modules = Modules)   
    return Modules

In [37]:
import numpy as np
import pandas as pd
from ConsensusFragment import *
#from MostIntenseFragmentNorm import *
def ConsensusSpectra(module,
                     AlignedFragments_mz_Mat,
                     AlignedFragmentsMat,
                     percentile_mz = 5,
                     percentile_Int = 10,
                     minSpectra = 3,
                     alpha = 0.01,
                     min_spectra = 3,
                     Columns_to_return = np.array([ 0, 3, 9, 10, 11, 17, 18, 19, 20, 21, 22])):
    N_Fragments = len(AlignedFragments_mz_Mat[:,0])    
    consensus_spectra = []
    for fragment in np.arange(N_Fragments):
        SpectraFragmentVec = AlignedFragments_mz_Mat[fragment, np.array(module)+1]
        SpectraIntensityVec = AlignedFragmentsMat[fragment, np.array(module)+1]
        consensus_spectra = ConsensusFragment(SpectraFragmentVec = SpectraFragmentVec,
                                              SpectraIntensityVec = SpectraIntensityVec,
                                              consensus_spectra = consensus_spectra,
                                              percentile_mz = percentile_mz,
                                              percentile_Int = percentile_Int,
                                              minSpectra = minSpectra,
                                              alpha = alpha)
    Columns = np.array(["median_mz(Da)",
                        "mz_std(Da)",
                        "p-Shapiro",
                        "N_spectra",
                        "t-student Confidence_interval(Da)",
                        "t-student Confidence_interval(ppm)",
                        "mean_Int_NoNorm",
                        "min_Int_NoNorm",
                        "max_Int_NoNorm",
                        "min_mz",
                        "max_mz",
                        "IQR_mz(ppm)",
                        "Q1_mz",
                        "Q3_mz",
                        "median_Int_NoNorm",
                        "Q1_Int_NoNorm",
                        "Q3_Int_NoNorm",
                        "mean_Int",
                        "min_Int",
                        "max_Int",
                        "median_Int",
                        "Int_Q1",
                        "Int_Q3"])        
    if len(consensus_spectra) == 0:
        return []
    consensus_spectra = MostIntenseFragmentNorm(consensus_spectra = consensus_spectra)
    consensus_spectraDF = pd.DataFrame(consensus_spectra, columns = Columns)
    consensus_spectraDF = consensus_spectraDF[Columns[Columns_to_return]].copy() 
    print('dataFrame')
    ShowDF(consensus_spectraDF)

    return consensus_spectraDF

In [ ]:
    AlignedSamplesMat, ms2_ids_inModules = ms2GaussFeatStats(All_SummMS2Table = All_SummMS2Table,
                                                             SamplesNames = SamplesNames,
                                                             Modules = Modules,
                                                             min_ms2_spectra = 4)        
    #This could become its own function for saving and formating 
    Write_ms2ids(features_ids = AlignedSamplesMat[:,11],
                 ms2_ids_inModules = ms2_ids_inModules,
                 AligningFolder = 'Alignedms2Features')   
    
    


In [83]:
date = datetime.datetime.now()
string_date = str(date)
string_date = string_date[:10].replace(':',"_")
string_date = string_date.replace(' ',"_")
string_date

'2026-02-17'

In [84]:
import pandas as pd
import os
import datetime
def Write_ms2ids_and_Consensus_ms2Spectra(feature_id,
                                          feature_module,
                                          consensus_spectraDF,
                                          All_FeaturesTable,
                                          sample_id_col = 6,
                                          ms2_spec_id_col = 0):
    date = datetime.datetime.now()
    string_date = str(date)
    string_date = string_date[:10].replace(':',"_")
    string_date = string_date.replace(' ',"_")
    AligningFolder = 'Alignedms2Features' + string_date
    if not os.path.exists(AligningFolder):
        os.mkdir(AligningFolder)
    FeatureTable = All_FeaturesTable[feature_module, :]
    feature_ms2s_table = FeatureTable[:, [ms2_spec_id_col, sample_id_col]]
    feature_ms2sDF = pd.DataFrame(feature_ms2s_table,columns = ['ms2_id','Sample_id'],dtype='int') 
    FeatureClusterLoc = AligningFolder +'/'+str(int(feature_id)) + '.csv'
    feature_ms2sDF.to_csv(FeatureClusterLoc)
    consensus_spectraLoc = AligningFolder +'/Consensus_ms2-spectra_'+str(int(feature_id)) + '.csv' 
    consensus_spectraDF.to_csv(consensus_spectraLoc)

In [85]:
import numpy as np
import pandas as pd
from ConsensusFragment import *
#from MostIntenseFragmentNorm import *
def ClosingModule(module,
                  Modules,
                  AlignedFragments_mz_Mat,
                  AlignedFragmentsMat,
                  Feature_module,
                  feature_id,
                  All_FeaturesTable,
                  sample_id_col = 6,
                  ms2_spec_id_col = 0,
                  percentile_mz = 5,
                  percentile_Int = 10,
                  minSpectra = 3,
                  alpha = 0.01,
                  min_spectra = 3):
    npFeature_module = np.array(Feature_module)
    if len(module) < min_spectra:
        return [Modules, feature_id]
    consensus_spectraDF = ConsensusSpectra(module = module,
                                           min_spectra = min_spectra,
                                           AlignedFragmentsMat = AlignedFragmentsMat,
                                           AlignedFragments_mz_Mat = AlignedFragments_mz_Mat,
                                           percentile_mz = percentile_mz,
                                           percentile_Int = percentile_Int)  
    if len(consensus_spectraDF) == 0:
        return [Modules, feature_id]    
    feature_module = npFeature_module[module].tolist()    
    Modules.append(feature_module)
    Write_ms2ids_and_Consensus_ms2Spectra(feature_id = feature_id,
                                          feature_module = feature_module,
                                          consensus_spectraDF = consensus_spectraDF,
                                          All_FeaturesTable = All_FeaturesTable,
                                          sample_id_col = sample_id_col,
                                          ms2_spec_id_col = ms2_spec_id_col)
    feature_id += 1
    return [Modules, feature_id]

In [86]:
import numpy as np
#from ConsensusSpectra import *
#from ClosingModule import *
def Update_ids_FeatureModules(Feature_module,
                              Feature_Modules,
                              AlignedFragmentsMat,
                              All_FeaturesTable,
                              AlignedFragments_mz_Mat,
                              sample_id_col = 6,
                              ms2_spec_id_col = 0,
                              percentile_mz = 5,
                              percentile_Int = 10,
                              feature_id = 0,
                              min_spectra = 3):    
    Modules=[]
    for module in Feature_Modules:        
        Modules, feature_id = ClosingModule(module = module,
                                            Modules = Modules,
                                            min_spectra = min_spectra,
                                            All_FeaturesTable = All_FeaturesTable,
                                            sample_id_col = sample_id_col,
                                            ms2_spec_id_col = ms2_spec_id_col,
                                            Feature_module = Feature_module,
                                            AlignedFragmentsMat = AlignedFragmentsMat,
                                            AlignedFragments_mz_Mat = AlignedFragments_mz_Mat,
                                            percentile_mz = percentile_mz,
                                            percentile_Int = percentile_Int,
                                            feature_id = feature_id)            
    return [Modules, feature_id]

In [89]:
from Retrieve_and_Join_ms2_for_feature import *
from AdjacencyList_ms2Fragments import *
from ms2_feat_modules import *
#from AligniningFragments_in_Feature import *
#from minimalAlignedFragmentsMat import *
from CosineMatrix import *
from AdjacencyList_from_matrix import *
from CommunityBlocks import *
#from OverlappingClustering import *
#from Update_ids_FeatureModules import *

def ms2_FeaturesDifferences(All_FeaturesTable,
                            Feature_module,
                            SamplesNames,
                            sample_id_col = 16,
                            ms2_spec_id_col = 15,
                            ms2Folder = 'ms2_spectra',
                            ToAdd = 'mzML',
                            cos_tol = 0.9,
                            Intensity_to_explain = 0.9,
                            min_spectra = 3,
                            Norm2One = False,
                            percentile = 10,
                            percentile_mz = 5,
                            percentile_Int = 10,
                            feature_id = 0):
    All_ms2, Spectra_idVec = Retrieve_and_Join_ms2_for_feature(All_FeaturesTable = All_FeaturesTable,
                                                               Feature_module = Feature_module,
                                                               SamplesNames = SamplesNames,
                                                               sample_id_col = sample_id_col,
                                                               ms2_spec_id_col = ms2_spec_id_col,
                                                               ms2Folder = ms2Folder,
                                                               ToAdd = ToAdd,
                                                               Norm2One = Norm2One)
    if len(All_ms2) == 0:
        return []
    Feature_module = np.array(Feature_module)[Spectra_idVec].tolist()
    AdjacencyListFragments, feat_ids = AdjacencyList_ms2Fragments(All_ms2 = All_ms2)
    N_features = len(Feature_module)
    Frag_Modules = ms2_feat_modules(AdjacencyList = AdjacencyListFragments,
                                    ms2_ids = feat_ids)
    AlignedFragmentsMat, AlignedFragments_mz_Mat = AligniningFragments_in_Feature(Frag_Modules = Frag_Modules,
                                                                                  All_ms2 = All_ms2,
                                                                                  N_features = N_features)   
    AlignedFragmentsMat, AlignedFragments_mz_Mat = minimalAlignedFragmentsMat(AlignedFragmentsMat = AlignedFragmentsMat,
                                                                              AlignedFragments_mz_Mat = AlignedFragments_mz_Mat,
                                                                              Intensity_to_explain = Intensity_to_explain,
                                                                              min_spectra = min_spectra)
    ShowDF(AlignedFragmentsMat)
    CosineMat = CosineMatrix(AlignedFragmentsMat = AlignedFragmentsMat,
                             N_features = N_features)
    AdjacencyList_Features, features_ids = AdjacencyList_from_matrix(CosineMat = CosineMat,
                                                                     N_ms2_spectra = N_features,
                                                                     cos_tol = cos_tol)
    Feature_Modules = CommunityBlocks(AdjacencyList_Features = AdjacencyList_Features)
    Modules = OverlappingClustering(Feature_Modules = Feature_Modules,
                                    CosineMat = CosineMat.copy(),
                                    percentile = percentile)    
    #print(Modules)
    #Modules, feature_id = Update_ids_FeatureModules(Feature_module = Feature_module,
    #                                                Feature_Modules = Modules,
    #                                                All_FeaturesTable = All_FeaturesTable,
    #                                                sample_id_col = sample_id_col,
    #                                                ms2_spec_id_col = ms2_spec_id_col,
    #                                                AlignedFragmentsMat = AlignedFragmentsMat,
    #                                                AlignedFragments_mz_Mat = AlignedFragments_mz_Mat,
    #                                                percentile_mz = percentile_mz,
    #                                                percentile_Int = percentile_Int,
    #                                                feature_id = feature_id,
    #                                                min_spectra = min_spectra)
    return [Modules, feature_id]

In [24]:
DataFolder=home+'/ms2_spectra_Summ-20260107'
SamplesList = os.listdir(DataFolder)
SamplesList.sort()
SamplesNames = [sample_name.replace('mzML-ms2Summary.xlsx','') for sample_name in SamplesList]



In [90]:
N_spectra = len(FeatureClustersTest)
ResultsFolder = home+'/ms2_spectra_Summ-20260107'
ms2Folder = home+'/ms2_spectra-20260107'
ms2_FeaturesDifferences(All_FeaturesTable= np.array(FeatureClustersTest),
                            Feature_module =  np.arange(N_spectra),
                            SamplesNames = SamplesNames,
                            sample_id_col = 1,
                            ms2_spec_id_col = 0,
                            ms2Folder = ms2Folder,
                            ToAdd = 'mzML',
                            cos_tol = 0.9,
                            Intensity_to_explain = 0.9,
                            min_spectra = 3,
                            Norm2One = True,
                            percentile = 10,
                            percentile_mz = 5,
                            percentile_Int = 10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135
0,92.0494,0.090239,0.0849794,0.0708033,0.0931118,0.0814,0.0661321,0.0685449,0.0888447,0.0986935,0.0762792,0.0831158,0.0758492,0.102893,0.0939572,0.0720235,0.0562226,0.0618182,0,0.071915,0,0.0610281,0.0878987,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0782167,0.0665849,0.064665,0.0695608,0.126053,0.0649184,0.0674471,0.109461,0.0629225,0.076716,0.0659599,0.0735124,0.0722373,0.076649,0.0643527,0.0710751,0.0653853,0.0625215,0.072537,0.0860477,0.0735409,0.064738,0.0887784,0.0708275,0.0609024,0.0650199,0.0591729,0.0629098,0.0682095,0.0773202,0.0753091,0.0603633,0.0861576,0.0650115,0.0618845,0.0729221,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.07945,0.0630933,0.0660582,0.075863,0.0643701,0.0609969,0.0622969,0.0655861,0.106403,0.074107
1,93.0574,0.0510359,0.0591371,0.0380464,0.0630814,0.0511349,0.05438,0.0396287,0.0480773,0,0,0,0.0541143,0.0547259,0.0374853,0.0583491,0.0509379,0,0,0,0,0.0571294,0.0478944,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0558354,0.0558368,0.0580273,0.0523071,0,0.0525925,0.054861,0,0.0494397,0.0638292,0.0503969,0.052816,0.0587787,0.0530345,0.0434679,0.0563732,0.0553464,0.0491609,0.0566848,0.0553388,0.0568474,0.0517653,0.051218,0.0501379,0.0577638,0.0523474,0.0585649,0.0569692,0.0548008,0.042004,0.0567826,0.0530797,0.0584728,0.0550342,0.0489815,0.0623166,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.054654,0.058197,0.0569296,0.0494821,0.0565216,0.0484509,0.0558638,0.0551754,0
2,99.0553,0.0521297,0.0729614,0.0678782,0.0729427,0.0455719,0.0446937,0.0551626,0.0578,0,0,0,0.0478194,0.0631596,0.0726069,0.066879,0.0512618,0.077649,0,0.0641882,0,0.0782004,0.0480595,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0695411,0.071333,0.066247,0.0771982,0.104743,0.0568803,0.061759,0.113464,0.0721912,0.0614093,0.0690343,0.0642885,0.0728765,0.0547081,0.0688797,0.0604512,0.0587975,0.0648334,0.0686267,0.0863933,0.0665977,0.0638196,0.0764575,0.0697795,0.0727719,0.0659367,0.0638546,0.0728443,0.0678584,0.0500166,0.0679105,0.0615849,0.0715912,0.0610049,0.0739526,0.0683251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0671676,0.064262,0.0555431,0.0657013,0.0615601,0.0637725,0.0748767,0.0727683,0
3,108.044,0.113258,0.10433,0.114139,0.129864,0.136532,0.147264,0.104496,0.128889,0.108432,0.136473,0.118095,0.0972221,0.103973,0.11437,0.112295,0.101851,0.130728,0,0.114614,0.15983,0.145143,0.0893025,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.130418,0.112119,0.126492,0.118722,0.136714,0.122502,0.114551,0.125036,0.11036,0.0986815,0.116141,0.122153,0.132572,0.136697,0.123568,0.11754,0.101472,0.117976,0.120403,0.097957,0.125265,0.118961,0.155207,0.111818,0.123156,0.133099,0.128222,0.118891,0.118337,0.114,0.111369,0.124609,0.101377,0.113404,0.116024,0.126214,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.111721,0.12051,0.119859,0.120699,0.133822,0.121809,0.0922543,0.110897,0.130673,0.0829982
4,128.107,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0501972,0.0636748,0.0688585,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0183817,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,138.091,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.761072,0.77218,0.747461,0.773517,0.829906,0.818215,0.808489,0.738171,0.789375,0.797157,0.759977,0.731581,0.796018,0.812658,0.755181,0.83429,0.790171,

[[[105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   61],
  [69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   131,
   128,
   129,
   130,
   132,
   133,
   126,
   127,
   125,
   134],
  [32, 33, 34, 35, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]],
 0]

In [9]:
import pandas as pd
import numpy as np
from AdjacencyListFeatures import *
from ms2_feat_modules import *
#from ms2_FeaturesDifferences import *
def ms2_SpectralSimilarityClustering(SummMS2_raw,
                                     SampleName = '',
                                     SamplesNames = [],
                                     mz_col = 1,
                                     RT_col = 2,
                                     RT_tol = 20,
                                     mz_Tol = 1e-2,
                                     sample_id_col = -1,
                                     ms2_spec_id_col = 0,
                                     ms2Folder = 'ms2_spectra',
                                     ToAdd = 'mzML',
                                     cos_tol = 0.9,
                                     Norm2One = False):
    if len(SamplesNames) == 0:
        SamplesNames = [SampleName]
    AdjacencyList, feat_ids = AdjacencyListFeatures(MS2_features = SummMS2_raw,
                                                    mz_col = mz_col,
                                                    RT_col = RT_col,
                                                    RT_tol = RT_tol,
                                                    mz_Tol = mz_Tol)
    RawModules = ms2_feat_modules(AdjacencyList = AdjacencyList,
                                  ms2_ids = feat_ids)
    Modules = []
    feature_id = 0
    for Feature_module in RawModules:
        Feature_Modules = ms2_FeaturesDifferences(All_FeaturesTable = SummMS2_raw,
                                                  Feature_module = Feature_module,
                                                  SamplesNames = SamplesNames,
                                                  sample_id_col = sample_id_col,
                                                  ms2_spec_id_col = ms2_spec_id_col,
                                                  ms2Folder = ms2Folder,
                                                  ToAdd = ToAdd,
                                                  cos_tol = cos_tol,
                                                  Norm2One = Norm2One,
                                                  feature_id = feature_id)
        Modules += Feature_Modules
    return Modules

In [10]:
import numpy as np
from ttest import *
def ms2GaussFeatStats(All_SummMS2Table, 
                      SamplesNames, 
                      Modules, 
                      min_ms2_spectra = 4):
    N_samples = len(SamplesNames)
    N_Features = len(Modules)
    AlignedSamplesMat = np.zeros((N_Features,
                                  N_samples+12))
    ms2_ids_inModules = []
    for feature_id in np.arange(N_Features,
                                dtype = 'int'):
        Feature_module = Modules[feature_id][0]
        ShowDF(Modules[feature_id][1])
        FeatureTable = All_SummMS2Table[Feature_module, :]
        AlignedSamplesMat[feature_id, 0] = np.mean(FeatureTable[:, 1])
        AlignedSamplesMat[feature_id, 1] = np.std(FeatureTable[:, 1])
        AlignedSamplesMat[feature_id, 2] = len(Feature_module)
        AlignedSamplesMat[feature_id, 3] = 0
        AlignedSamplesMat[feature_id, 4] = 0
        AlignedSamplesMat[feature_id, 8] = np.mean(FeatureTable[:, 2])
        AlignedSamplesMat[feature_id, 9] = np.min(FeatureTable[:, 2])
        AlignedSamplesMat[feature_id, 10] = np.max(FeatureTable[:, 2])
        AlignedSamplesMat[feature_id, 11] = int(feature_id)
        ms2_ids_inModules.append(FeatureTable[:, [0, 6]])
        Samples_ids = np.array(FeatureTable[:, 6],
                               dtype = 'int')
        Samples_ids = set(list(Samples_ids))
        Samples_ids = np.array(list(Samples_ids))
        AlignedSamplesMat_loc = Samples_ids + 12
        AlignedSamplesMat[feature_id, AlignedSamplesMat_loc] = 1
    SpectralFilter = np.where(AlignedSamplesMat[:, 2] > min_ms2_spectra)[0]
    AlignedSamplesMat = AlignedSamplesMat[SpectralFilter, :] 
    NSamplesVec = list(AlignedSamplesMat[:, 2])
    t_refVec = list(map(ttest, NSamplesVec))    
    AlignedSamplesMat[:, 6] = t_refVec*AlignedSamplesMat[:, 1] / np.sqrt(NSamplesVec)
    AlignedSamplesMat[:, 7] = AlignedSamplesMat[:, 6] / AlignedSamplesMat[:, 0]*1e6    
    AlignedSamplesMat = AlignedSamplesMat[AlignedSamplesMat[:, 0].argsort()]    
    return [AlignedSamplesMat, ms2_ids_inModules]

In [12]:
import numpy as np
import pandas as pd
import os
import datetime
from JoiningSummMS2 import *
#from ms2_SpectralSimilarityClustering import *
#from ms2GaussFeatStats import *
#from Write_ms2ids import *
def ms2_SamplesAligment(ResultsFolder,
                        mz_min = 254,
                        mz_max = 255,
                        RT_min = 0,
                        RT_max = 2000,
                        RT_tol = 30,
                        mz_Tol = 2e-3,
                        cos_tol = 0.8,
                        min_N_ms2_spectra = 3,
                        ToReplace = 'mzML-ms2Summary.xlsx',
                        ms2Folder = 'ms2_spectra',
                        ToAdd = 'mzML',
                        saveAlignedTable = False,
                        name = "SamplesAligment",
                        Norm2One = True):
    home = os.getcwd()
    #ResultsFolder = home+'/'+ResultsFolderName
    All_SummMS2Table, SamplesNames = JoiningSummMS2(ResultsFolder = ResultsFolder,
                                                    mz_min = mz_min-3*mz_Tol,
                                                    mz_max = mz_max+3*mz_Tol,
                                                    RT_min = RT_min,
                                                    RT_max = RT_max,
                                                    ToReplace = ToReplace)
    Modules = ms2_SpectralSimilarityClustering(SummMS2_raw = All_SummMS2Table,
                                               SamplesNames = SamplesNames,
                                               mz_col = 1,
                                               RT_col = 2,
                                               RT_tol = RT_tol,
                                               mz_Tol = mz_Tol,
                                               sample_id_col = 6,
                                               ms2_spec_id_col = 0,
                                               ms2Folder = ms2Folder,
                                               ToAdd = ToAdd,
                                               cos_tol = cos_tol,
                                               Norm2One = Norm2One)

    AlignedSamplesMat, ms2_ids_inModules = ms2GaussFeatStats(All_SummMS2Table = All_SummMS2Table,
                                                             SamplesNames = SamplesNames,
                                                             Modules = Modules,
                                                             min_ms2_spectra = 4)        
    #This could become its own function for saving and formating 
    Write_ms2ids(features_ids = AlignedSamplesMat[:,11],
                 ms2_ids_inModules = ms2_ids_inModules,
                 AligningFolder = 'Alignedms2Features')    
    Columns = ['mz_(Da)',
               'mz_std_(Da)',
               'N_ms2-spectra',
               'mean CosSim',
               'std CosSim',
               'Normal test',
               'mz_ConfidenceInterval_(Da)',
               'mz_ConfidenceInterval_(ppm)',
               'RT_(s)',
               'min_RT_(s)',
               'max_RT_(s)',
               'feat_id']
    Columns = Columns + SamplesNames 

    AlignedSamplesDF = pd.DataFrame(AlignedSamplesMat,columns = Columns)
    if saveAlignedTable:
        date = datetime.datetime.now()
        string_date = str(date)
        string_date = string_date[:16].replace(':',"_")
        string_date = string_date.replace(' ',"_")
        name = name+"_"+string_date+'.xlsx'
        AlignedSamplesDF.to_excel(name)
    return AlignedSamplesDF

In [60]:
#from ms2_SamplesAligment import *
ResultsFolder = home+'/ms2_spectra_Summ-20260107'
ms2Folder = home+'/ms2_spectra-20260107'
AlignedSamplesDF = ms2_SamplesAligment(ResultsFolder = ResultsFolder,
                                       mz_min = 259,
                                       mz_max = 259.2,
                                       RT_min = 0,
                                       RT_max = 2000,
                                       RT_tol = 30,
                                       mz_Tol = 2e-3,
                                       cos_tol = 0.9,
                                       min_N_ms2_spectra = 3,
                                       ToReplace = 'mzML-ms2Summary.xlsx',
                                       ms2Folder = ms2Folder,
                                       ToAdd = 'mzML',
                                       saveAlignedTable = False,
                                       name = "SamplesAligment",
                                       Norm2One = True)

,0,1,2
0,55.0542,55.0542,55.0541
1,73.0647,73.0648,73.0646
2,83.049,83.0491,83.049
3,101.06,101.06,101.06
4,111.044,111.044,111.044
5,129.054,129.055,129.055
6,141.091,141.09,141.091
7,143.07,143.069,143.07
8,159.02,159.02,159.021
9,169.086,169.086,169.086


[[np.float64(55.05415612815244), 0, 0, 3, 0, 0, np.float64(0.04377219424188957), np.float64(0.04112454718682671), np.float64(0.04637283129100694), np.float64(55.05414107194269), np.float64(55.0541714082284), np.float64(0.3061256996480268), np.float64(55.054147763591466), np.float64(55.05416461708353), np.float64(0.04388971925675329), np.float64(0.04216148671304918), np.float64(0.04544166427816182)], [np.float64(73.06471159901307), 0, 0, 3, 0, 0, np.float64(0.04937599199031741), np.float64(0.04356160626769983), np.float64(0.05670297447825755), np.float64(73.06465585223353), np.float64(73.06474731359071), np.float64(0.6954364697011036), np.float64(73.06468062857999), np.float64(73.06473144044509), np.float64(0.045594500077010984), np.float64(0.04432394144619151), np.float64(0.05253729657779009)], [np.float64(83.04904634693138), 0, 0, 3, 0, 0, np.float64(0.03863848890379382), np.float64(0.03672003951681013), np.float64(0.040989556802178674), np.float64(83.04903836396393), np.float64(83.04

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,55.0542,0,0,3,0,0,0.0437722,0.0411245,0.0463728,55.0541,55.0542,0.306126,55.0541,55.0542,0.0438897,0.0421615,0.0454417
1,73.0647,0,0,3,0,0,0.049376,0.0435616,0.056703,73.0647,73.0647,0.695436,73.0647,73.0647,0.0455945,0.0443239,0.0525373
2,83.049,0,0,3,0,0,0.0386385,0.03672,0.0409896,83.049,83.0491,0.17063,83.049,83.0491,0.0375569,0.0370339,0.0397023
3,101.06,0,0,3,0,0,0.206088,0.191138,0.221496,101.06,101.06,0.222747,101.06,101.06,0.204945,0.196315,0.21529
4,111.044,0,0,3,0,0,0.599085,0.547328,0.665317,111.044,111.044,0.536239,111.044,111.044,0.562901,0.553168,0.626911
5,129.055,0,0,3,0,0,0.238029,0.217183,0.25874,129.054,129.055,0.211068,129.054,129.055,0.238366,0.225127,0.2511
6,141.091,0,0,3,0,0,0.025311,0.0222394,0.0292111,141.09,141.091,1.58819,141.09,141.091,0.0232396,0.0226144,0.0269718
7,143.07,0,0,3,0,0,0.0349155,0.0254379,0.0442052,143.069,143.07,2.94283,143.07,143.07,0.0353849,0.0291681,0.0408976
8,159.02,0,0,3,0,0,0.110985,0.0818579,0.137273,159.02,159.021,0.969816,159.02,159.021,0.118081,0.0954417,0.130076
9,169.086,0,0,3,0,0,0.17312,0.151327,0.199978,169.086,169.086,0.857736,169.086,169.086,0.160458,0.154751,0.185158


dataFrame


,median_mz(Da),N_spectra,min_mz,max_mz,IQR_mz(ppm),mean_Int,min_Int,max_Int,median_Int,Int_Q1,Int_Q3
0,55.0542,3,55.0541,55.0542,0.306126,6.92488,6.50602,7.33631,6.94347,6.67006,7.18899
1,73.0647,3,73.0647,73.0647,0.695436,7.81142,6.89156,8.97056,7.21317,7.01217,8.31154
2,83.049,3,83.049,83.0491,0.17063,6.11271,5.80921,6.48466,5.94161,5.85886,6.28102
3,101.06,3,101.06,101.06,0.222747,32.6037,30.2385,35.0413,32.4229,31.0576,34.0594
4,111.044,3,111.044,111.044,0.536239,94.7769,86.5888,105.255,89.0524,87.5126,99.179
5,129.055,3,129.054,129.055,0.211068,37.6568,34.359,40.9334,37.7101,35.6157,39.7247
6,141.091,3,141.09,141.091,1.58819,4.00426,3.51833,4.62127,3.67656,3.57767,4.26701
7,143.07,3,143.069,143.07,2.94283,5.52372,4.02435,6.99338,5.59799,4.61447,6.47011
8,159.02,3,159.02,159.021,0.969816,17.5581,12.9501,21.7169,18.6808,15.0991,20.5784
9,169.086,3,169.086,169.086,0.857736,27.3881,23.9404,31.6371,25.3849,24.4821,29.2925


,0
0,55.0542
1,73.0648
2,83.0491
3,101.06
4,111.044
5,112.047
6,129.055
7,141.091
8,143.07
9,169.086


[]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [188]:
N_Fragments = len(AlignedFragments_mz_Mat[:,0])
for module in Modules:
    consensus_spectraDF = ConsensusSpectra(module = module,
                                         N_Fragments = N_Fragments,
                                         AlignedFragmentsMat = AlignedFragmentsMat,
                                         AlignedFragments_mz_Mat = AlignedFragments_mz_Mat,
                                         percentile_mz = 5,
                                         percentile_Int = 10)

    #ShowDF(consensus_spectra)
    ShowDF(consensus_spectraDF)
    #print(consensus_spectra[maxIntFragLoc, 0])
    #ShowDF(100 * consensus_spectra[:, 6] / consensus_spectra[maxIntFragLoc, 6])
    #ShowDF(100 * consensus_spectra[:, 8] / consensus_spectra[maxIntFragLoc, 6])

,median_mz(Da),N_spectra,min_mz,max_mz,IQR_mz(ppm),mean_Int,min_Int,max_Int,median_Int,Int_Q1,Int_Q3
0,138.091,53,138.091,138.091,0.989605,100,93.945,106.705,100.274,97.149,103.984
1,254.09,26,254.089,254.091,2.59919,15.0767,0,37.8121,0,0,24.4377
2,254.175,53,254.175,254.176,1.38783,79.1636,71.6422,87.4017,80.1319,73.0591,84.836
3,255.178,9,255.175,255.179,8.14525,2.68986,0,9.44617,0,0,0


,median_mz(Da),N_spectra,min_mz,max_mz,IQR_mz(ppm),mean_Int,min_Int,max_Int,median_Int,Int_Q1,Int_Q3
0,92.0494,66,92.0492,92.0496,1.47225,8.07847,6.82148,10.1844,7.93187,7.22086,8.77855
1,99.0552,61,99.055,99.0556,1.9867,6.67894,3.49752,8.42372,7.29644,6.32145,8.02018
2,108.044,67,108.044,108.045,1.02851,13.1166,11.2428,15.2587,13.2508,12.4798,14.1893
3,147.079,57,147.079,147.08,1.5446,8.31492,0,11.0145,9.87633,7.3138,10.4793
4,156.011,68,156.011,156.011,0.484676,100,96.5116,103.33,98.8136,97.9235,101.53
5,157.014,4,157.014,157.015,1.5492,0.323104,0,0,0,0,0
6,160.087,43,160.087,160.087,2.53154,5.78856,0,10.1437,8.16148,0,9.37932
7,188.082,52,188.081,188.082,2.31926,11.2654,0,17.4641,14.1056,9.4297,15.4104
8,254.059,64,254.059,254.06,1.81334,41.2526,34.7177,49.6599,44.9953,39.7166,46.7412


,median_mz(Da),N_spectra,min_mz,max_mz,IQR_mz(ppm),mean_Int,min_Int,max_Int,median_Int,Int_Q1,Int_Q3
0,254.09,14,254.089,254.091,1.3337,100,100,100,100,100,100
